In [54]:
import pandas as pd
import numpy as np

In [ ]:
events = pd.read_csv("raw_data\Eventfile7_Sept2001.csv")
events = events.dropna(axis=1, how="all")

# events.head()

In [56]:
events = events.copy()

def parse_date(x):
    # Handle NaN
    if pd.isna(x):
        return pd.NaT

    # Turn into a clean string
    s = str(x).strip()

    # If it came from Excel as a float, strip .0
    if s.endswith(".0"):
        s = s[:-2]

    # Keep only digits
    s = "".join(ch for ch in s if ch.isdigit())

    if len(s) == 0:
        return pd.NaT

    # Decide format by length
    if len(s) == 6:     # yymmdd
        return pd.to_datetime(s, format="%y%m%d", errors="coerce")
    elif len(s) == 8:   # yyyymmdd
        return pd.to_datetime(s, format="%Y%m%d", errors="coerce")
    else:
        # Treat unexpected length as missing
        return pd.NaT
    
events["event_date"] = events["DATES"].apply(parse_date)

events[events["event_date"] < "1975-01-01"]
events[events["event_date"] > "2025-01-01"]
events.shape

(1099, 18)

## Data Cleaning

In [57]:
# Drop rows where original DATES is missing
events = events.dropna(subset=["DATES"]).copy()

# Drop rows where parsed event_date is missing
events = events.dropna(subset=["event_date"]).copy()

# Remove rows with missing CERTAIN values
events = events.dropna(subset=["CERTAIN"]).copy()

# Reset Index
events = events.reset_index(drop=True)

# Check remaining missing values
events.isna().sum().sort_values(ascending=False)

Sort          0
CASE          0
DECISION      0
TARJCASE      0
COMPANY       0
PERMNO        0
COUNTRY       0
PRODUCT       0
CERTAIN       0
DATES         0
EVENT         0
NP            0
ACQ           0
JV            0
PE            0
Semicon       0
steel92       0
event_date    0
dtype: int64

In [58]:
events["EVENT"].value_counts()

core_events = ["PET", "ITCP", "DOCP", "DOCF", "ITCF"]

ad_events = events[events["EVENT"].isin(core_events)].copy()
ad_events

,Sort,CASE,DECISION,TARJCASE,COMPANY,PERMNO,COUNTRY,PRODUCT,CERTAIN,DATES,EVENT,NP,ACQ,JV,PE,Semicon,steel92,event_date
0,1.0,731003,A,0.0,AMSTAR CORP.,10479,CANADA,SUGAR AND SYRUPS,0.0,790430,PET,na,na,na,na,0.0,0.0,1979-04-30
1,2.0,731003,A,0.0,AMSTAR CORP.,10479,CANADA,SUGAR AND SYRUPS,0.0,791108,DOCF,na,na,na,na,0.0,0.0,1979-11-08
2,3.0,731003,A,0.0,AMSTAR CORP.,10479,CANADA,SUGAR AND SYRUPS,0.0,800326,ITCF,na,na,na,na,0.0,0.0,1980-03-26
3,6.0,731012,A,1.0,SCM CORP.,20474,JAPAN,Portable Electric Typewriters,0.0,790409,PET,na,na,na,na,0.0,0.0,1979-04-09
4,7.0,731012,A,1.0,SCM CORP.,20474,JAPAN,Portable Electric Typewriters,0.0,800101,ITCP,na,na,na,na,0.0,0.0,1980-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062,1087.0,731053-86,T,1.0,U.S. STEEL CORP.,15069,UK,Carbon steel products,1.0,820111,PET,na,na,na,na,0.0,0.0,1982-01-11
1063,1088.0,731053-86,T,1.0,BETHLEHEM STEEL CORP.,10786,UK,Carbon steel products,1.0,820218,ITCP,na,na,na,na,0.0,0.0,1982-02-18
1064,1089.0,731053-86,T,1.0,U.S. STEEL CORP.,15069,UK,Carbon steel products,1.0,820228,ITCP,na,na,na,na,0.0,0.0,1982-02-28
1065,1090.0,731053-86,T,1.0,BETHLEHEM STEEL CORP.,10786,UK,Carbon steel products,1.0,820812,DOCP,na,na,na,na,0.0,0.0,1982-08-12


In [59]:
bad_perm = events[~events["PERMNO"].astype(str).str.fullmatch(r"\d+")]
bad_perm[["CASE", "COMPANY", "PERMNO"]]

,CASE,COMPANY,PERMNO
364,731278,ITT Grinnell,12570/45356
366,731278,ITT Grinnell,12570/45356
368,731278,ITT Grinnell,12570/45356
370,731278,ITT Grinnell,12570/45356
372,731278,ITT Grinnell,12570/45356


In [60]:
def clean_permno(x):
    s = str(x).strip()
    if "/" in s:
        s = s.split("/")[0]   # keep only the first PERMNO
    return s

events["PERMNO"] = events["PERMNO"].apply(clean_permno)
events["PERMNO"] = pd.to_numeric(events["PERMNO"], errors="coerce").astype("Int64")

events["PERMNO"].apply(lambda x: str(x).isdigit()).value_counts()

PERMNO
True    1071
Name: count, dtype: int64

In [61]:
# Verify counts per case

events.groupby("CASE")["EVENT"].nunique().value_counts()

EVENT
5    69
6    23
4     3
3     2
2     2
Name: count, dtype: int64

In [62]:
order_map = {"PET":1,"ITCP":2,"DOCP":3,"DOCF":4,"ITCF":5}

events["event_order"] = events["EVENT"].map(order_map)

violations = []
for case, group in events.groupby("CASE"):
    grp_sorted = group.sort_values("event_date")
    if not grp_sorted["event_order"].dropna().is_monotonic_increasing:
        violations.append(case)

violations


['731368']

In [63]:
events[events["CASE"] == "731368"].sort_values("event_date")

,Sort,CASE,DECISION,TARJCASE,COMPANY,PERMNO,COUNTRY,PRODUCT,CERTAIN,DATES,EVENT,NP,ACQ,JV,PE,Semicon,steel92,event_date,event_order
566,587.0,731368,A,1.0,ZENITH ELECTRONICS CORP.,18067,JAPAN,Color Picture Tubes,1.0,841212,TARJ,1,0,0,0,0.0,0.0,1984-12-12,NaN
567,588.0,731368,A,1.0,ZENITH ELECTRONICS CORP.,18067,JAPAN,Color Picture Tubes,0.0,861126,PET,na,na,na,na,0.0,0.0,1986-11-26,1.0
568,589.0,731368,A,1.0,ZENITH ELECTRONICS CORP.,18067,JAPAN,Color Picture Tubes,0.0,870112,ITCP,na,na,na,na,0.0,0.0,1987-01-12,2.0
569,590.0,731368,A,1.0,ZENITH ELECTRONICS CORP.,18067,JAPAN,Color Picture Tubes,1.0,870625,DOCP,na,na,na,na,0.0,0.0,1987-06-25,3.0
570,591.0,731368,A,1.0,ZENITH ELECTRONICS CORP.,18067,JAPAN,Color Picture Tubes,0.0,870721,ITCF,na,na,na,na,0.0,0.0,1987-07-21,5.0
571,592.0,731368,A,1.0,ZENITH ELECTRONICS CORP.,18067,JAPAN,Color Picture Tubes,1.0,871113,DOCF,na,na,na,na,0.0,0.0,1987-11-13,4.0
572,593.0,731368,A,1.0,ZENITH ELECTRONICS CORP.,18067,JAPAN,Color Picture Tubes,1.0,871226,TARJ,1,0,0,0,0.0,0.0,1987-12-26,NaN
573,594.0,731368,A,1.0,ZENITH ELECTRONICS CORP.,18067,JAPAN,Color Picture Tubes,1.0,891211,TARJ,1,0,0,0,0.0,0.0,1989-12-11,NaN


## Initiate DataFrames & Obtain CSV

In [64]:
core = ["PET", "ITCP", "DOCP", "DOCF", "ITCF"]
ad_events = events[events["EVENT"].isin(core)].copy()
ad_events.shape


(889, 19)

In [65]:
tarj_events = events[events["EVENT"] == "TARJ"].copy()
tarj_events.shape

(177, 19)

In [ ]:
# ad_events.to_csv("AD_events_clean.csv", index=False)
# tarj_events.to_csv("TARJ_events_clean.csv", index=False)